# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data_df = pd.read_csv("weather_data_results.csv")
weather_data_df.head()


,city,latitude,longitude,temperature,humidity,cloudiness,wind speed
0,nikolskoye,59.70,30.79,27.32,39,few clouds,24.61
1,tuktoyaktuk,69.45,-133.04,21.20,85,light snow,28.86
2,nizhniy tsasuchey,50.52,115.13,12.18,69,clear sky,3.38
3,ancud,-41.87,-73.82,60.80,59,few clouds,6.93
4,vaini,-21.20,-175.20,80.60,88,broken clouds,10.29


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = []
weights = []

for index, row  in weather_data_df.iterrows():
    
    try:
        locations.append([row.loc["latitude"], row.loc["longitude"]])
        weights.append(float(row["humidity"]))
        
    except:
        print(f"row[2], row[3], row[5] ignored")
        
# print(locations)
# print(weights)

In [4]:
gmaps.configure(api_key=g_key)

fig = gmaps.figure()
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=1)
fig = gmaps.figure(layout={
        'width': '800px',
        'height': '500px',
        'padding': '3px',
        'border': '1px solid black'
})
heat_layer = gmaps.heatmap_layer(locations, weights=weights)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', padding='3px', width='800px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
vacation_spots = []

for index, row  in weather_data_df.iterrows():
    
    if row.loc["temperature"]  >= 70 and row.loc["temperature"] <= 80:
        if row.loc["wind speed"] <= 10:
            if row.loc["cloudiness"] == "clear sky":
                vacation_spots.append(row)
        
print(vacation_spots)
    

[city           hambantota
latitude             6.12
longitude           81.12
temperature          78.8
humidity               88
cloudiness      clear sky
wind speed           2.24
Name: 132, dtype: object, city               pecos
latitude           31.42
longitude        -103.49
temperature        70.05
humidity              37
cloudiness     clear sky
wind speed          4.05
Name: 169, dtype: object, city               azare
latitude           11.68
longitude          10.19
temperature        77.16
humidity              14
cloudiness     clear sky
wind speed          7.87
Name: 170, dtype: object, city               avera
latitude           33.19
longitude         -82.53
temperature        79.38
humidity              47
cloudiness     clear sky
wind speed          3.36
Name: 177, dtype: object, city           trincomalee
latitude              8.57
longitude            81.23
temperature          71.04
humidity                95
cloudiness       clear sky
wind speed            2.15

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_details = {}
hotel_map_details = []
for spot in vacation_spots:

    query_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=" + str(spot.loc["latitude"]) + "," + str(spot.loc["longitude"]) + "&radius=5000&type=hotel&key=" + g_key
    
    hotels = requests.get(query_url)
    hotels_json = hotels.json()
    include = True
    
    for key in hotels_json["results"]:
        if include and key['name'] != key['vicinity']:
            hotel_details = {"location": (key['geometry']['location']['lat'], 
                                      key['geometry']['location']['lng']), 'name': key['name'],
                                      'city': key['vicinity'] 
                            }
            hotel_map_details.append(hotel_details)
            include = False

            
print(hotel_map_details)        

[{'location': (6.130587499999999, 81.1251296), 'name': 'Bungalow 63', 'city': 'No 63 Barrak Street, Hambantota'}, {'location': (31.4284957, -103.4860338), 'name': 'Knights Inn Pecos', 'city': '1000 East 3rd Street, Pecos'}, {'location': (11.674166, 10.191111), 'name': 'Guaranty Trust Bank PLC', 'city': 'No. 231, Egbeda - Idimu Road, Alimosho L.g.a, Egbeda, Lagos'}, {'location': (33.19403539999999, -82.5270728), 'name': 'Georgia Life and Health Agency Inc', 'city': 'Avera'}, {'location': (8.570033, 81.215834), 'name': 'Welcombe Hotel', 'city': "66, Orr's Hill Centre Rd, Trincomalee"}, {'location': (20.6846539, -105.2391801), 'name': 'Container Inn Puerto Vallarta', 'city': 'Carretera a, Puerto Vallarta - Tepic 4634, Guadalupe Victoria, Puerto Vallarta'}, {'location': (30.372352, -88.5327102), 'name': 'Regency Inn', 'city': '3800 Hospital Road, Pascagoula'}, {'location': (18.2948077, 83.89617199999999), 'name': 'Jeans Park', 'city': 'GT Road, Chinna Bharatham Colony, Srikakulam'}, {'loca

In [7]:

hotel_locations = [hotel["location"] for hotel in hotel_map_details]
# print(hotel_locations)
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Address</dt><dd>{city}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**hotel) for hotel in hotel_map_details]
# print(hotel_info)

fig = gmaps.figure()
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=1)
fig = gmaps.figure(layout={
        'width': '800px',
        'height': '500px',
        'padding': '3px',
        'border': '1px solid black'
})
heat_layer = gmaps.heatmap_layer(locations, weights=weights)
fig.add_layer(heat_layer)
marker_layer = gmaps.marker_layer(hotel_locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', padding='3px', width='800px'))

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map